In [1]:
import pandas as pd
from pandas.io.json import json_normalize
from yahoo_fantasy_sports import YahooFantasySports

In [2]:
credentials_file = 'oauth.json'
yfs = YahooFantasySports(credentials_file)

Enter code: 2uyk6y


In [3]:
# league settings summary
url = 'http://fantasysports.yahooapis.com/fantasy/v2/leagues;league_keys=nfl.l.591106'

In [4]:
# league settings detail
url = 'https://fantasysports.yahooapis.com/fantasy/v2/league/nfl.l.591106/settings'

In [5]:
r = yfs.session.get(url, params={'format': 'json'})
r.status_code

200

In [6]:
# PPR enabled
ppr_enabled = int(r.json()['fantasy_content']['league'][1]
                  ['settings'][0]['stat_categories']
                  ['stats'][7]['stat']['enabled'])

ppr_enabled

1

In [7]:
# PPR value
ppr_value = int(r.json()['fantasy_content']['league'][1]
                ['settings'][0]['stat_modifiers']
                ['stats'][5]['stat']['value'])

ppr_value

1

In [8]:
# function that structures json player data in a dataframe

def clean_player_data(players):
    # create a list of all player dataframes
    player_list = []

    # loop through all players in the json response
    for i in range(len(players)):
        index = str(i)
        try:
            # create a list of all player attributes
            player_data = []
            for attribute in players[index]['player'][0]:
                try:
                    player_data.append(json_normalize(attribute))
                except:
                    continue
                    
            # total points is at a different path which is why it is not included in the for loop above
            total_points = float(players[index]['player'][1]['player_points']['total'])
            player_data.append(json_normalize({'total_points':total_points}))

            # add one row with current player's data to a list
            player_list.append(pd.concat(player_data, axis=1))
        except:
            continue

    # store all player data in one dataframe
    df = pd.concat(player_list, ignore_index=True)
    
    return df

In [9]:
# available players in league
league = 'nfl.l.591106'

# can only query yahoo player api in chunks of 25
chunks = []
start = 0

# get player data in chunks of 25 and then combine all chunks in one dataframe
while start < 200:
    l = ['https://fantasysports.yahooapis.com/fantasy/v2/league/', league,
         '/players;sort=AR;sort_type=season;sort_season=2016;status=A;start=',
         str(start), '/stats']
    
    url = ''.join(l)
    
    r = yfs.session.get(url, params={'format': 'json'})
    
    players = r.json()['fantasy_content']['league'][1]['players']
    
    chunks.append(clean_player_data(players))
    
    start += 25

yahoo_data = pd.concat(chunks, ignore_index=True)
yahoo_data['yahoo_rank'] = yahoo_data.index + 1

In [20]:
# keep relevant columns of yahoo data
cols = ['display_position', 'name.full', 'status', 'total_points', 'yahoo_rank']
yahoo_data = yahoo_data[cols]

yahoo_data.head()

,display_position,name.full,status,total_points,yahoo_rank
0,QB,Aaron Rodgers,NaN,0.0,1
1,QB,Matt Ryan,NaN,0.0,2
2,QB,Drew Brees,NaN,0.0,3
3,RB,David Johnson,NaN,0.0,4
4,QB,Andrew Luck,PUP-P,0.0,5


In [11]:
# get boris chen data for PPR
boris = 'https://docs.google.com/spreadsheets/d/1ZX8xNT6ObBw3v0p21YW7oOatZw1ZWLpwGuicC86aAeU/export'
boris_data = pd.read_excel(boris, sheetname='PPR')

name_split = boris_data['Player.Name'].str.rsplit(n=1, expand=True)
name_split = name_split.rename(columns={0:'name.full', 1:'team'})

boris_data['name.full'] = name_split['name.full']

boris_data.head()

,Rank,Player.Name,Tier,Position,Bye,Best,Worst,Avg.Rank,Std.Dev,ADP,vs..ADP,name.full
0,1,David Johnson ARI,1,RB1,8,1,3,1.3,0.5,1,0,David Johnson
1,2,Le'Veon Bell PIT,1,RB2,9,1,2,1.7,0.5,2,0,Le'Veon Bell
2,3,Antonio Brown PIT,1,WR1,9,2,7,3.6,0.8,3,0,Antonio Brown
3,4,Odell Beckham Jr. NYG,1,WR2,8,3,13,5.5,1.7,6,2,Odell Beckham Jr.
4,5,LeSean McCoy BUF,1,RB3,6,3,13,6.0,2.5,7,2,LeSean McCoy


In [21]:
# final dataframe with yahoo and boris data combined
data = pd.merge(yahoo_data, boris_data[['name.full', 'Rank', 'Tier']], on='name.full', how='left')

In [22]:
data.head()

,display_position,name.full,status,total_points,yahoo_rank,Rank,Tier
0,QB,Aaron Rodgers,NaN,0.0,1,30.0,5.0
1,QB,Matt Ryan,NaN,0.0,2,62.0,10.0
2,QB,Drew Brees,NaN,0.0,3,40.0,7.0
3,RB,David Johnson,NaN,0.0,4,1.0,1.0
4,QB,Andrew Luck,PUP-P,0.0,5,66.0,11.0


In [23]:
data[data['Tier'] == 1]

,display_position,name.full,status,total_points,yahoo_rank,Rank,Tier
3,RB,David Johnson,NaN,0.0,4,1.0,1.0
11,RB,Le'Veon Bell,Q,0.0,12,2.0,1.0
18,WR,Antonio Brown,NaN,0.0,19,3.0,1.0
22,WR,Odell Beckham Jr.,NaN,0.0,23,4.0,1.0
23,RB,LeSean McCoy,NaN,0.0,24,5.0,1.0
32,WR,Julio Jones,NaN,0.0,33,6.0,1.0
